### Basic MLP example with synapx

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

# Import .././ directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from examples.utils.data import split_dataset, DataLoader, DataLoaderCallback
from examples.utils.train import Trainer, Evaluator

In [ ]:
supported_engines = ['synapx', 'torch']

In [ ]:
seed = 37
engine_str = 'synapx'

In [ ]:
epochs = 50
batch_size = 4
lr = 0.1

In [ ]:
if engine_str == 'synapx':
    import synapx as engine
    from synapx import nn, optim
elif engine_str == 'torch':
    import torch as engine
    from torch import nn, optim

torch.manual_seed(seed)
print("Engine:", engine_str)

In [ ]:
# make up a dataset
from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=150, noise=0.1)

y = y*2 - 1 # make y be -1 or 1
y = np.where(y > 0, 1, 0).tolist() # 0 or 1
# visualize in 2D
plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

In [ ]:
class MultiLayerPerceptron(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.mlp = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            if module.weight.shape[0] == 1: # last layer
                nn.init.xavier_uniform_(module.weight)
                with engine.no_grad():
                    module.weight *= 0.1 # make logits layer less confident
            else:
                nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
            nn.init.zeros_(module.bias)
            
    def forward(self, x:engine.Tensor):
        return self.mlp(x)

model = MultiLayerPerceptron()
print("MLP Model:", model)
print("MLP Trainable Parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
class Transform(DataLoaderCallback):
    
    def __call__(self, data_loader:'DataLoader', X_batch:np.ndarray, y_batch:np.ndarray):
        x = engine.tensor(X_batch)
        y = engine.tensor(y_batch)
        return x, y
    
transform_cb = Transform()

split = False
if split:
    train, val, _ = split_dataset(X, y, test_split=0.2, shuffle=True)
    print("Train samples:", len(train), "| Val samples:", len(val))
    train_loader = DataLoader(train[0], train[1], batch_size, transform=transform_cb)
    val_loader = DataLoader(val[0], val[1], len(val[0]), transform=transform_cb) # 1 batch of all val samples
    print("Train batches:", len(train_loader), "| Val batches:", len(val_loader))
else:
    train_loader = DataLoader(np.array(X, dtype=np.float32), np.array(y, dtype=np.float32), batch_size, transform=transform_cb)
    print("Train batches:", len(train_loader))

In [ ]:
loss_fn = nn.MSELoss() # nn.BCEWithLogitsLoss() could be used as well
optimizer = optim.SGD(model.parameters(), lr=lr)

trainer = Trainer(model, engine)
evaluator = Evaluator(accuracy=True, mode=Evaluator.BINARY)
trainer.compile(loss_fn, optimizer, evaluator)
history = trainer.fit(train_loader, epochs=epochs, validation_loader=None if not split else val_loader)

In [ ]:
trainer.plot(['accuracy', 'loss'])

In [ ]:
# visualize decision boundary
model.eval()
h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Xmesh = (np.c_[xx.ravel(), yy.ravel()]).astype(np.float32)

inputs = engine.tensor(Xmesh)
scores = model(inputs)
Z = np.array([s.detach().numpy() > 0.5 for s in scores])
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())